In [1]:
import pandas as pd

# Đọc file Excel
file_path = "SuperStore.xlsx"
df = pd.read_excel(file_path)

# Chọn cột cần thiết
df = df[['Order ID', 'Order Date', 'Ship Date', 'Customer ID', 'Customer Name', 'Segment', 
         'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 
         'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit']]

# Xử lý bảng DimDate
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date'] = pd.to_datetime(df['Ship Date'])
df['Year'] = df['Order Date'].dt.year
df['Month'] = df['Order Date'].dt.month
df['Day'] = df['Order Date'].dt.day

# Xuất file CSV
df.to_csv("Cleaned_SuperStore.csv", index=False)
print("✅ Dữ liệu đã được xử lý và lưu vào 'Cleaned_SuperStore.csv'")


✅ Dữ liệu đã được xử lý và lưu vào 'Cleaned_SuperStore.csv'


In [3]:
import pyodbc
import pandas as pd

# Thông tin kết nối SQL Server 
server = r'DESKTOP-URB07L7\SQLEXPRESS'  
database = 'SuperStoreDB1'
username = 'sa'  
password = '123' 

# Kết nối SQL Server
conn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')
cursor = conn.cursor()

# Đọc dữ liệu đã xử lý từ file CSV
df = pd.read_csv("Cleaned_SuperStore.csv")

# Insert dữ liệu vào bảng DimCustomer
for index, row in df.iterrows():
    cursor.execute("""
        IF NOT EXISTS (SELECT 1 FROM DimCustomer WHERE CustomerID = ?)
        INSERT INTO DimCustomer (CustomerID, CustomerName, Segment) 
        VALUES (?, ?, ?)""",
        row['Customer ID'], row['Customer ID'], row['Customer Name'], row['Segment'])

# Insert dữ liệu vào bảng DimProduct
for index, row in df.iterrows():
    cursor.execute("""
        IF NOT EXISTS (SELECT 1 FROM DimProduct WHERE ProductID = ?)
        INSERT INTO DimProduct (ProductID, ProductName, Category, SubCategory) 
        VALUES (?, ?, ?, ?)""",
        row['Product ID'], row['Product ID'], row['Product Name'], row['Category'], row['Sub-Category'])

# Insert dữ liệu vào bảng DimRegion
for index, row in df.iterrows():
    cursor.execute("""
        IF NOT EXISTS (SELECT 1 FROM DimRegion WHERE Country = ? AND State = ? AND City = ?)
        INSERT INTO DimRegion (Country, State, City, PostalCode) 
        VALUES (?, ?, ?, ?)""",
        row['Country'], row['State'], row['City'], row['Country'], row['State'], row['City'], row['Postal Code'])

# Insert dữ liệu vào bảng DimDate
for index, row in df.iterrows():
    cursor.execute("""
        IF NOT EXISTS (SELECT 1 FROM DimDate WHERE OrderDate = ?)
        INSERT INTO DimDate (OrderDate, ShipDate, Year, Month, Day) 
        VALUES (?, ?, ?, ?, ?)""",
        row['Order Date'], row['Order Date'], row['Ship Date'], row['Year'], row['Month'], row['Day'])

# Insert dữ liệu vào bảng FactSales
for index, row in df.iterrows():
    cursor.execute("""
        INSERT INTO FactSales (OrderID, CustomerID, ProductID, RegionID, DateID, Sales, Profit, Quantity, Discount) 
        VALUES (?, ?, ?, 
            (SELECT TOP 1 RegionID FROM DimRegion WHERE Country = ? AND State = ? AND City = ?),
            (SELECT TOP 1 DateID FROM DimDate WHERE OrderDate = ?),
            ?, ?, ?, ?)""",
        row['Order ID'], row['Customer ID'], row['Product ID'],
        row['Country'], row['State'], row['City'], row['Order Date'],
        row['Sales'], row['Profit'], row['Quantity'], row['Discount'])

# Commit và đóng kết nối
conn.commit()
cursor.close()
conn.close()

print("✅ Dữ liệu đã được nhập thành công vào SQL Server!")


✅ Dữ liệu đã được nhập thành công vào SQL Server!
